## Leitura e Pré-Processamento dos Dados

### 1. Criação dos Arquivos CSV com dados da Selic e do Fundo de Interesse

#### 1.1. Selic

In [88]:
'''
import pandas as pd

codigo_serie = '11'
data_inicio_selic = '04/01/2021'
data_fim_selic = '30/09/2024'

url_selic = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio_selic}&dataFinal={data_fim_selic}'

dados_selic = pd.read_csv(url_selic, sep=';', encoding='ISO-8859-1')

dados_selic = dados_selic.rename(columns={'valor': 'selic_ao_dia'})

dados_selic.set_index('data', inplace=True)

dados_selic.index = pd.to_datetime(pd.to_datetime(dados_selic.index, format='mixed').date)

Dados_Selic_CSV = dados_selic.to_csv('Dados_Selic_CSV.csv')
'''


"\nimport pandas as pd\n\ncodigo_serie = '11'\ndata_inicio_selic = '04/01/2021'\ndata_fim_selic = '30/09/2024'\n\nurl_selic = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio_selic}&dataFinal={data_fim_selic}'\n\ndados_selic = pd.read_csv(url_selic, sep=';', encoding='ISO-8859-1')\n\ndados_selic = dados_selic.rename(columns={'valor': 'selic_ao_dia'})\n\ndados_selic.set_index('data', inplace=True)\n\ndados_selic.index = pd.to_datetime(pd.to_datetime(dados_selic.index, format='mixed').date)\n\nDados_Selic_CSV = dados_selic.to_csv('Dados_Selic_CSV.csv')\n"

#### 1.2. Ibovespa

In [89]:
import pandas as pd
import yfinance as yf
import pandas_datareader.data as pdr 
from datetime import datetime
''''
data_inicial = datetime(2021, 1, 4)

data_final = datetime(2024, 10, 1)

benchmark = '^BVSP'

ibov = yf.download(benchmark, start=data_inicial, end=data_final, interval='1d')

ibov = pd.DataFrame(ibov)

Dados_Ibov_CSV = ibov.to_csv('Dados_Ibov_CSV.csv')
'''

"'\ndata_inicial = datetime(2021, 1, 4)\n\ndata_final = datetime(2024, 10, 1)\n\nbenchmark = '^BVSP'\n\nibov = yf.download(benchmark, start=data_inicial, end=data_final, interval='1d')\n\nibov = pd.DataFrame(ibov)\n\nDados_Ibov_CSV = ibov.to_csv('Dados_Ibov_CSV.csv')\n"

#### 1.2. SICREDI FIA PETROBRÁS (Variação Diária)

In [90]:
'''
import pandas as pd
import zipfile 
import requests

pd.options.display.float_format = '{:4f}'.format # para melhor visualização dos dados (casas decimais muito grandes)

# 2021 - 2024
anos = range(2021, 2025)  
meses = range(1, 13)  

anos_meses = [(ano, f"{mes:02d}") for ano in anos for mes in meses if (ano == 2024 and mes <= 9) or (ano < 2024)]

# Usar Nome associado ao CNPJ:

cadastro_fundos = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',
                              sep = ';',
                              encoding = 'ISO-8859-1')

cadastro_fundos = cadastro_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

cadastro_fundos = cadastro_fundos.drop_duplicates()

base_dados2 = pd.DataFrame()  # DF vazio 

for ano, mes in anos_meses:

    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip"

    paginaweb = requests.get(url)

    with open(f"inf_diario_fi_{ano}{mes}.zip", 'wb') as arquivo_cvm:

        arquivo_cvm.write(paginaweb.content)

    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano}{mes}.zip")

    dados_fundos = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep=';',    )

    # processar dados ordenados cronologicamente
    dados_fundos['DT_COMPTC'].sort_values(ascending=True)

    dados_fundos = pd.merge(dados_fundos, cadastro_fundos, how="left", left_on="CNPJ_FUNDO", right_on="CNPJ_FUNDO")

    dados_fundos = dados_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

    # fundo SICREDI
    fundo_sicredi_petr = dados_fundos[dados_fundos['CNPJ_FUNDO'].str.contains('08.336.054/0001-34', na=False)]

    # Concatenando dados em um dataframe (base de dados)
    base_dados2 = pd.concat([base_dados2, fundo_sicredi_petr], ignore_index=True)

base_dados2.to_csv('Dados_Inicio_Fim_Dia_SICREDIPETR.csv', index=False)
'''

'\nimport pandas as pd\nimport zipfile \nimport requests\n\npd.options.display.float_format = \'{:4f}\'.format # para melhor visualização dos dados (casas decimais muito grandes)\n\n# 2021 - 2024\nanos = range(2021, 2025)  \nmeses = range(1, 13)  \n\nanos_meses = [(ano, f"{mes:02d}") for ano in anos for mes in meses if (ano == 2024 and mes <= 9) or (ano < 2024)]\n\n# Usar Nome associado ao CNPJ:\n\ncadastro_fundos = pd.read_csv(\'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv\',\n                              sep = \';\',\n                              encoding = \'ISO-8859-1\')\n\ncadastro_fundos = cadastro_fundos[[\'CNPJ_FUNDO\', \'DENOM_SOCIAL\']]\n\ncadastro_fundos = cadastro_fundos.drop_duplicates()\n\nbase_dados2 = pd.DataFrame()  # DF vazio \n\nfor ano, mes in anos_meses:\n\n    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip"\n\n    paginaweb = requests.get(url)\n\n    with open(f"inf_diario_fi_{ano}{mes}.zip", \'wb\') as arqu

### 2. Tratamento Inicial dos Dados

In [91]:
import pandas as pd

#### 2.1.DataFrame com o Fundo

In [92]:
dados_bruto_diario = pd.read_csv('Dados_Inicio_Fim_Dia_SICREDIPETR.csv')

dados_sicredi_cru = dados_bruto_diario[['DT_COMPTC', 'VL_QUOTA']]

# Datas para o formato DateTime

dados_sicredi_cru['DT_COMPTC'] = pd.to_datetime(dados_sicredi_cru['DT_COMPTC'])

# Novo DF para os Cálculos

dados_sicredi = dados_sicredi_cru[['DT_COMPTC', 'VL_QUOTA']]

# Datas como índice

dados_sicredi.set_index('DT_COMPTC', inplace=True)

C:\Users\Guima\AppData\Local\Temp\ipykernel_14692\1840305206.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### 2.2. DataFrame com a Selic 

In [93]:
dados_selic = pd.read_csv("Dados_Selic_CSV.csv", decimal=',')

# Tratar dados - Data e Valor de str para datetime e float

dados_selic['selic_ao_dia'] = pd.to_numeric(dados_selic['selic_ao_dia'])

dados_selic.columns = ['Data'] + list(dados_selic.columns[1:])

dados_selic['Data'] = dados_sicredi_cru['DT_COMPTC']

#### 2.3. DataFrame com Ibovespa

In [94]:
import numpy as np

dados_ibov = pd.read_csv('Dados_Ibov_CSV.csv')

dados_ibov = dados_ibov.rename(columns={'Adj Close':'Adj_Close'})

dados_ibov = dados_ibov.rename(columns={'Date':'Data'})

dados_ibov = dados_ibov[['Data', 'Adj_Close']]

dados_ibov['Data'] = pd.to_datetime(dados_ibov['Data'])

dados_ibov['Adj_Close'] = pd.to_numeric(dados_ibov['Adj_Close'])

dados_ibov.set_index('Data', inplace=True)

dados_ibov['Retorno_Diario_Ibov(%)'] = dados_ibov['Adj_Close'].pct_change()*100

dados_ibov['Log_Return_Ibov(%)'] = np.log(dados_ibov['Adj_Close']/dados_ibov['Adj_Close'].shift(1))*100

#### 2.3. Combinação dos DataFrames em um só

In [95]:
dados_combinados1 = pd.merge(dados_sicredi,dados_selic,how='outer',left_index=True,right_on='Data')

dados_combinados1.set_index('Data',inplace=True)

dados_combinados1 = pd.merge(dados_combinados1,dados_ibov,how='outer',left_index=True, right_index=True)

## Cálculo dos Indicadores Técnicos

In [96]:
import pandas as pd
import numpy as np

### 1. Retornos Diários e Retornos Diários(%)

In [97]:
dados_combinados1['Retornos_Diarios'] = dados_combinados1['VL_QUOTA'].pct_change()

dados_combinados1['Retornos_Diarios(%)'] = dados_combinados1['Retornos_Diarios']*100

### 2. *Log_Returns* e *Log_Returns(%)*

In [98]:
dados_combinados1['Log_Return(%)'] = np.log(dados_combinados1['VL_QUOTA']/dados_combinados1['VL_QUOTA'].shift(1))*100

### 3. Volatilidade Diária Anualizada e Volatilidade Diária(%)

In [99]:
dados_combinados1['Volatilidade_Dia'] = ((dados_combinados1['Retornos_Diarios'].rolling(window=22)).std())*np.sqrt(252)

dados_combinados1['Volatilidade_Dia(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=22)).std())*np.sqrt(252)

### 4. Índice Sharpe Diário (fechamento)

In [100]:
dados_combinados1['Sharpe/Dia'] = (dados_combinados1['Retornos_Diarios(%)'] - dados_combinados1['selic_ao_dia']) / dados_combinados1['Volatilidade_Dia(%)']

## Visualizações

In [101]:
import plotly.express as px
import matplotlib.pyplot as plt

### 1. Rentabilidade Diária

In [102]:
graph_return = px.line(dados_combinados1, x=dados_combinados1.index, y='Retornos_Diarios(%)', title='Rentabilidade Diária (%)')

graph_return.show()

### 2. *Log_Returns*

In [103]:
graph_LOG_return = px.line(dados_combinados1, x=dados_combinados1.index, y='Log_Return(%)', title='Gráfico com Log Retorno (diário)')

graph_LOG_return.show()

### 3. Rentabilidade Diária contra Selic e Ibov

In [104]:
graph_fundreturn_selic_ibovreturn = px.line(dados_combinados1, x=dados_combinados1.index, y=['Retornos_Diarios(%)',
                                                                                             'selic_ao_dia',
                                                                                             'Retorno_Diario_Ibov(%)'])

graph_fundreturn_selic_ibovreturn.show()

### 4. Volatilidade Diária

In [105]:
graph_volatility_daily = px.line(dados_combinados1, 
                                 x=dados_combinados1.index, 
                                 y='Volatilidade_Dia(%)', 
                                 title='Volatilidade Diária do Fundo (%)')

graph_volatility_daily.show()

## Cálculo dos Indicadores Técnicos em Diferentes Janelas de Tempo

### 1. Retornos em 1 mês, 3 meses, 6 meses, 12 meses, 24 meses e 36 meses

In [106]:
# 1 mês
dados_combinados1['Retorno_1M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=21)*100 # 21 dias / mês

# 3 meses
dados_combinados1['Retorno_3M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=65)*100 # 65 dias / 03 meses

# 6 meses
dados_combinados1['Retorno_6M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=123)*100 # 123 dias / 06 meses

# 12 meses
dados_combinados1['Retorno_12M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=252)*100 # 252 dias / 12 meses

# 24 meses
dados_combinados1['Retorno_24M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=503)*100 # 503 dias / 24 meses

# 36 meses
dados_combinados1['Retorno_36M(%)'] = dados_combinados1['VL_QUOTA'].pct_change(periods=752)*100 # 752 dias / 36 meses

### 2. Volatilidade nas Janelas de Tempo Definidas

In [107]:
# 1 mês
dados_combinados1['Volatilidade_Mensal(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=21)).std())*np.sqrt(252)

# 3 meses
dados_combinados1['Volatilidade_3M(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=65)).std())*np.sqrt(252)

# 6 meses
dados_combinados1['Volatilidade_6M(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=123)).std())*np.sqrt(252)

# 12 meses
dados_combinados1['Volatilidade_12M(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=252)).std())*np.sqrt(252)

# 24 meses
dados_combinados1['Volatilidade_24M(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=503)).std())*np.sqrt(252)

# 36 meses
dados_combinados1['Volatilidade_36M(%)'] = ((dados_combinados1['Retornos_Diarios(%)'].rolling(window=752)).std())*np.sqrt(252)

### 3. Índice Sharpe nas Janelas de Tempo

#### 3.1. Selic para diferentes períodos

In [111]:
'''def retorno_acumulado(df, dias):
    return (df['selic_ao_dia']/100 + 1).rolling(window=dias).apply(np.prod, raw=True) - 1

# Retorno acumulado em 30 dias (aqui como exemplo para um dataframe maior)
dados_combinados1['retorno_30_dias'] = retorno_acumulado(dados_combinados1, 22)*100

dados_combinados1['retorno_65_dias'] = retorno_acumulado(dados_combinados1, 65)*100


# Retorno acumulado em 252 dias (aproximadamente 1 ano útil)
dados_combinados1['retorno_252_dias'] = retorno_acumulado(dados_combinados1, 252)
dados_combinados1'''

"def retorno_acumulado(df, dias):\n    return (df['selic_ao_dia']/100 + 1).rolling(window=dias).apply(np.prod, raw=True) - 1\n\n# Retorno acumulado em 30 dias (aqui como exemplo para um dataframe maior)\ndados_combinados1['retorno_30_dias'] = retorno_acumulado(dados_combinados1, 22)*100\n\ndados_combinados1['retorno_65_dias'] = retorno_acumulado(dados_combinados1, 65)*100\n\n\n# Retorno acumulado em 252 dias (aproximadamente 1 ano útil)\ndados_combinados1['retorno_252_dias'] = retorno_acumulado(dados_combinados1, 252)\ndados_combinados1"

In [112]:
dados_combinados1['selic_ao_mes'] = dados_combinados1['selic_ao_dia'].rolling(21).sum()

dados_combinados1['selic_3M'] = dados_combinados1['selic_ao_dia'].rolling(65).sum()

dados_combinados1['selic_6M'] = dados_combinados1['selic_ao_dia'].rolling(123).sum()

dados_combinados1['selic_12M'] = dados_combinados1['selic_ao_dia'].rolling(252).sum()

dados_combinados1['selic_24M'] = dados_combinados1['selic_ao_dia'].rolling(503).sum()

dados_combinados1['selic_36M'] = dados_combinados1['selic_ao_dia'].rolling(752).sum()

#### 3.2. Índice Sharpe

In [113]:
dados_combinados1['IS_1M'] = (dados_combinados1['Retorno_1M(%)'] - dados_combinados1['selic_ao_mes']) / dados_combinados1['Volatilidade_Mensal(%)']

dados_combinados1['IS_3M'] = (dados_combinados1['Retorno_3M(%)'] - dados_combinados1['selic_3M']) / dados_combinados1['Volatilidade_3M(%)']

dados_combinados1['IS_6M'] = (dados_combinados1['Retorno_6M(%)'] - dados_combinados1['selic_6M']) / dados_combinados1['Volatilidade_6M(%)']

dados_combinados1['IS_12M'] = (dados_combinados1['Retorno_12M(%)'] - dados_combinados1['selic_12M']) / dados_combinados1['Volatilidade_12M(%)']

dados_combinados1['IS_24M'] = (dados_combinados1['Retorno_24M(%)'] - dados_combinados1['selic_24M']) / dados_combinados1['Volatilidade_24M(%)']

dados_combinados1['IS_36M'] = (dados_combinados1['Retorno_36M(%)'] - dados_combinados1['selic_36M']) / dados_combinados1['Volatilidade_36M(%)']

## Organização do DataFrame

In [114]:
dados_combinados1 = dados_combinados1[['Retorno_Diario_Ibov(%)', 'Log_Return_Ibov(%)',
                                       'selic_ao_dia', 'selic_ao_mes', 'selic_3M', 'selic_6M', 'selic_12M', 'selic_24M', 'selic_36M',
                                       'VL_QUOTA', 'Retornos_Diarios', 'Retornos_Diarios(%)', 'Log_Return(%)',
                                       'Volatilidade_Dia', 'Volatilidade_Dia(%)', 'Sharpe/Dia',
                                       'Retorno_1M(%)', 'Volatilidade_Mensal(%)', 'IS_1M',
                                       'Retorno_3M(%)', 'Volatilidade_3M(%)', 'IS_3M',
                                       'Retorno_6M(%)', 'Volatilidade_6M(%)', 'IS_6M',
                                       'Retorno_12M(%)', 'Volatilidade_12M(%)', 'IS_12M',
                                       'Retorno_24M(%)', 'Volatilidade_24M(%)', 'IS_24M',
                                       'Retorno_36M(%)', 'Volatilidade_36M(%)', 'IS_36M']]

In [115]:
dados_combinados1

,Retorno_Diario_Ibov(%),Log_Return_Ibov(%),selic_ao_dia,selic_ao_mes,selic_3M,selic_6M,selic_12M,selic_24M,selic_36M,VL_QUOTA,...,IS_6M,Retorno_12M(%),Volatilidade_12M(%),IS_12M,Retorno_24M(%),Volatilidade_24M(%),IS_24M,Retorno_36M(%),Volatilidade_36M(%),IS_36M
Data,,,,,,,,,,,,,,,,,,,,,
2021-01-04,NaN,NaN,0.007469,NaN,NaN,NaN,NaN,NaN,NaN,0.962805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,0.560907,0.559340,0.007469,NaN,NaN,NaN,NaN,NaN,NaN,1.000309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-06,0.526744,0.525362,0.007469,NaN,NaN,NaN,NaN,NaN,NaN,1.001292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-07,1.756347,1.741102,0.007469,NaN,NaN,NaN,NaN,NaN,NaN,1.032193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-08,2.559120,2.526922,0.007469,NaN,NaN,NaN,NaN,NaN,NaN,1.036161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24,1.216224,1.208888,0.040168,0.828262,2.556142,4.856252,10.556461,23.237697,33.375551,3.497258,...,0.218246,26.817578,26.879189,1.825168,84.275329,45.543087,1.340217,254.893629,57.344354,3.862945
2024-09-25,-0.431308,-0.432241,0.040168,0.829160,2.557040,4.856252,10.549350,23.227077,33.392032,3.522128,...,0.299612,26.936725,26.881149,1.825299,97.939275,45.095140,1.656768,259.800446,57.340349,3.948501
2024-09-26,1.082182,1.076368,0.040168,0.830058,2.557938,4.856252,10.542239,23.216457,33.408513,3.448314,...,0.251440,27.075119,26.872153,1.826175,96.938693,45.117269,1.634014,246.758540,57.368924,3.718913


## Inserção de Outros Indicadores Financeiros

### 1. β

### 2. Tracking Error

### 3. Sortino Ratio

### 4. Information Ratio

### 5. Treynor Index

## StreamLit Code Model

In [116]:
'''import streamlit as st

st.title('DashBoard - SICREDI FIA PETROBRÁS')

st.subheader('Tabela com Dados Financeiros Compilados')

st.dataframe(dados_combinados1)

st.subheader('Visualização dos dados')

st.markdown('#### Gráfico de Rentabilidade Diária(%)')

st.plot'''

"import streamlit as st\n\nst.title('DashBoard - SICREDI FIA PETROBRÁS')\n\nst.subheader('Tabela com Dados Financeiros Compilados')\n\nst.dataframe(dados_combinados1)\n\nst.subheader('Visualização dos dados')\n\nst.markdown('#### Gráfico de Rentabilidade Diária(%)')\n\nst.plot"